# RNA-seq training 03/12/2024

## Materials

RNA-seq

Paper describing the data:

https://pubmed.ncbi.nlm.nih.gov/30899034/

Full list of samples:

https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=SRP132189&o=acc_s%3Aa&s=SRR6671775,SRR6671776,SRR6671777,SRR6671757,SRR6671758,SRR6671759 


reference genome:

https://www.ncbi.nlm.nih.gov/assembly/GCA_015534855.1 








## Setup

In [ ]:
mkdir rna_seq_2024
cd rna_seq_2024

In [ ]:
# Check conda and python are installed on your machine
conda list

In [ ]:
# if conda is not found, download with: 
wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh

sha256sum Miniconda3-py39_4.12.0-Linux-x86_64.sh                       # you should see a long hash code if conda has been downloaded correctly
bash Miniconda3-py39_4.12.0-Linux-x86_64.sh -b -p $HOME/miniconda      # follow the instructions to install, close and reopen your command line when finished

In [ ]:
# Create a conda environment to store RNA-seq tools
conda create -y -n rnaseq 

## Get data from SRA server

In [ ]:
# Install SRA toolkit to conda env
conda install -n rnaseq -y sra-tools -c bioconda
conda list -n rnaseq

In [ ]:
# Create directory for raw reads
mkdir raw_reads
cd raw_reads

In [ ]:
# Obtain fastq files from Sequence Read Archive
for file in SRR6671757 SRR6671758 SRR6671759 SRR6671775 SRR6671776 SRR6671777; do
    conda run -n rnaseq fasterq-dump $file
    echo $file
done

In [ ]:
# Zip reads
gzip *.fastq

## Read Quality Control

In [ ]:
# Install FastQC and MultiQC to conda env
conda install -y fastqc multiqc -c bioconda

In [ ]:
# Return to home directory and make new dir for fastQC reports
cd ..
mkdir qc_reports

In [ ]:
# Generate a fastQC report for each file
for file in raw_reads/*.fastq.gz; do
    fastqc $file -o qc_reports/
done

In [ ]:
# Aggregate reports with MultiQC
multiqc qc_reports/

## Read Filtering

In [ ]:
# Install trimmomatic to conda env
conda install -n rnaseq -y trimmomatic -c bioconda

In [ ]:
# Return to home directory and make new dir for trimmed reads
mkdir trimmed_paired trimmed_unpaired
wget https://raw.githubusercontent.com/gwickh/MMB_RNA-seq/refs/heads/main/NexteraPE-PE.fa


In [ ]:
# Trim reads with trimmomatic
for infile in raw_reads/*_1.fastq.gz; do
    base=$(basename ${infile} _1.fastq.gz)
    trimmomatic PE \
        ${infile} raw_reads/${base}_2.fastq.gz \
        trimmed_paired/${base}_1-trimmed-paired.fastq.gz trimmed_unpaired/${base}_1-trimmed-unpaired.fastq.gz \
        trimmed_paired/${base}_2-trimmed-paired.fastq.gz trimmed_unpaired/${base}_2-trimmed-unpaired.fastq.gz \
        ILLUMINACLIP:NexteraPE-PE.fa:2:30:10 LEADING:3 TRAILING:3 MINLEN:36 SLIDINGWINDOW:4:15
    echo ${base};
done

## Read Alignment

In [ ]:
# Install bowtie2 to conda env
conda install -n rnaseq -y bowtie2 -c bioconda

In [27]:
# Make new directory for alignments
mkdir aligned_reads alignment_index

In [ ]:
# Download a reference genome to the bowtie2-alignment directory and and build a bowtie index 
curl https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/015/534/855/GCA_015534855.1_ASM1553485v1/GCA_015534855.1_ASM1553485v1_cds_from_genomic.fna.gz -o ecoli_K12_ref.fna.gz
gunzip ecoli_K12_ref.fna
bowtie2-build ecoli_K12_ref.fna alignment_index/ecoli_K12_ref

In [ ]:
# Align reads against reference using Bowtie2
for infile in trimmed_paired/*1-trimmed-paired.fastq.gz; do
    base=$(basename ${infile} _1-trimmed-paired.fastq.gz)
    bowtie2 -x alignment_index/ecoli_K12_ref \
        -1 ${infile} -2 trimmed_paired/${base}_2-trimmed-paired.fastq.gz \
        -S aligned_reads/${base}.sam
    echo ${base};
done

## Alignment-Free Read Quantification

In [ ]:
# Install salmon to conda env
conda install -n rnaseq -y salmon -c bioconda

In [ ]:
# Make new directory for read quantification
mkdir quantified_transcripts

In [ ]:
# Build Salmon index 
curl https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/015/534/855/GCA_015534855.1_ASM1553485v1/GCA_015534855.1_ASM1553485v1_cds_from_genomic.fna.gz -o ecoli_K12_ref.fna.gz
gunzip ecoli_K12_ref.fna
salmon index -t ecoli_K12_ref.fna -i ecoli_K12_ref

In [ ]:
# Perform alignment-free read quantification
for infile in trimmed_paired/*_1-trimmed-paired.fastq.gz; do
    base=$(basename ${infile} _1-trimmed-paired.fastq.gz);
    salmon quant --libType A \
        -i ecoli_K12_ref \
        -o quantified_transcripts/${base} \
        -p 8 \
        --validateMappings \
        -1 ${infile} -2 trimmed_paired/${base}_2-trimmed-paired.fastq.gz
    echo ${infile};
done

# You now have a quantified transcriptome for each sample. 
# Open the .sf files in a text editor to inspect the calculated expression of each gene.

In [ ]:
# For performing read quantification with .sam files
for filename in aligned_reads/*.sam; do
    salmon quant --libType A \
        -t ecoli_K12_ref.fna  \
        -a $filename \
        -o quantified_transcripts/$(basename ${filename} .sam)_quant;
    echo $filename;
done